In [2]:
import magenta
import note_seq
import tensorflow
import os
import fluidsynth
import time

print(magenta.__version__)
print(tensorflow.__version__)


2024-04-17 00:21:56.163644: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-04-17 00:21:56.163698: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/aymliu/miniconda3/envs/magenta/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/home/aymliu/miniconda3/envs/magenta/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' req

2.1.4
2.9.1


In [3]:
tensorflow.compat.v1.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
os.environ['SDL_AUDIODRIVER']='dsp'

In [5]:
import sf2_loader as sf

from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel

#bass_vae = TrainedModel(configs.CONFIG_MAP['cat-mel_2bar_small'], batch_size=512, checkpoint_dir_or_path = '/tmp/music_vae/train/model.ckpt-6000')

print("done")

done


In [59]:
loader = sf.sf2_loader()



loader.load("./soundFonts/SQ_008_Rock_Bass.sf2")
loader.load("./soundFonts/The_Definitive_Perfect_Drums_Soundfount_V1___1-12_.sf2")

loader.change_sfid(1)
#print(loader)

#loader.get_all_instrument_names()


In [7]:
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.models.shared import sequence_generator_bundle
from note_seq.protobuf import generator_pb2
from note_seq.protobuf import music_pb2
from note_seq import constants



In [18]:
from magenta.models.drums_rnn import drums_rnn_sequence_generator



In [21]:

bundle = sequence_generator_bundle.read_bundle_file('/tmp/drums_rnn.mag')
generator_map = drums_rnn_sequence_generator.get_generator_map()
drums_rnn = generator_map['drum_kit'](checkpoint=None, bundle=bundle)
drums_rnn.initialize()

INFO:tensorflow:Restoring parameters from /tmp/tmp7j21p9tn/model.ckpt


In [20]:

bundle = sequence_generator_bundle.read_bundle_file('/tmp/melody_rnn2.mag')
generator_map = melody_rnn_sequence_generator.get_generator_map()
melody_rnn = generator_map['lookback_rnn'](checkpoint=None, bundle=bundle)
melody_rnn.initialize()

INFO:tensorflow:Restoring parameters from /tmp/tmphw65irtd/model.ckpt


In [52]:
def find_sf(instrument):
    loader.change_sfid(1)
    instrument_found = False
    print(loader.sfid_list)
    
    for i in range(len(loader.sfid_list)):
    
        if instrument in loader.get_current_instrument():
            return
        else:
            loader.change_sfid(i+1)

In [23]:
def generate_primer_sequence(primer_notes, primer_tempo, instrument):
    
    primer_seq = music_pb2.NoteSequence()
    
    start = 0
    
    primer_seq.tempos.add(qpm=primer_tempo)
    
    for note in primer_notes:
        
        if "Drum" in instrument:
            primer = note_seq.DrumTrack(
            [frozenset(pitches) for pitches in primer_notes]
            )
            
            primer_seq = primer.to_sequence(qpm=primer_tempo)
        else:
            primer_seq.notes.add(pitch=note['pitch'], start_time=start, end_time=start+(note['length']/(primer_tempo/60)), velocity=80)
            
            start = start+(note['length']/(primer_tempo/60))
    
            #print(start)
    

    return primer_seq

In [13]:
primer_notes = [{'pitch': 50, 'length': 1}, {'pitch': 48, 'length': 2}]

primer_drums = [(36,38), (), (36,)]


primer_tempo = 180

primer_seq = generate_primer_sequence(primer_notes, primer_tempo, "Bass")

note_seq.plot_sequence(primer_seq)
    

Loading BokehJS ...

In [34]:
#primer_notes is list of dictionaries with pitch and length (in beats) attributes

def generate_clip(instrument, temperature, length, primer_notes=None, tempo=None):
    
    find_sf(instrument)
    
    if primer_notes is not None:
        
        primer_seq = generate_primer_sequence(primer_notes, tempo, instrument)
            
    num_steps = length*16      
    
    last_end_time = (max(note.end_time for note in primer_seq.notes)
                  if primer_seq.notes else 0)
    
    seconds_per_step = 60/tempo/melody_rnn.steps_per_quarter
    total_seconds = num_steps * seconds_per_step
    
    generator_options = generator_pb2.GeneratorOptions()
    generator_options.args['temperature'].float_value = temperature
    generate_section = generator_options.generate_sections.add(
      start_time=last_end_time + seconds_per_step,
      end_time=total_seconds)
    
    if "Drum" not in instrument:
    
    
        sequence = melody_rnn.generate(primer_seq, generator_options)
    
    else: 
        
        sequence = drums_rnn.generate(primer_seq, generator_options)
        
    print(loader)
    return sequence
                
        
        

In [15]:
def loop_generate(instrument, temperature, length, loops, primer_notes=None, tempo=None, sequence=None, seconds_total=0, counter=0):
    
    #base case
    if loops == 0:
        return sequence
    
    #finds soundfont
    find_sf(instrument)    
        
    #checks for length to produce (first will be four, then six bc need to chop primer off so it doesn't repeat)
    if sequence is not None:
            
        num_steps = (length+2)*16   
    
    else:
        #creates sequence if it does not exist
        sequence = music_pb2.NoteSequence()
        sequence.tempos.add(qpm=tempo)
        sequence.total_time = 0
        
        num_steps = length*16
        
    #generates primer note sequence from primer notes
    if primer_notes is not None:
        
        primer_seq = generate_primer_sequence(primer_notes, tempo, instrument)
        
    #figures out how long primer is and subtracts it from length to find how much to generate
    
    last_end_time = (max(note.end_time for note in primer_seq.notes)
                  if primer_seq.notes else 0)
    
    seconds_per_step = 60/tempo/melody_rnn.steps_per_quarter
    total_seconds = num_steps * seconds_per_step
    
    generator_options = generator_pb2.GeneratorOptions()
    generator_options.args['temperature'].float_value = temperature
    generate_section = generator_options.generate_sections.add(
      start_time=last_end_time + seconds_per_step,
      end_time=total_seconds)
    
    #generates sequence including primer
    
    newSequenceFull = melody_rnn.generate(primer_seq, generator_options)
    
    twoSecsFromStart = 32*seconds_per_step
    
    sequenceToAdd = music_pb2.NoteSequence()
    
    j=0
    
    #chops off primer if it already exists in the larger sequence
    if counter != 0:
        for note in newSequenceFull.notes:

            if j==0:
                if note.start_time>twoSecsFromStart:
                    sequenceToAdd.notes.add(pitch=0, start_time=0, end_time=note.start_time-twoSecsFromStart, velocity=80)
                    sequenceToAdd.notes.add(pitch=note.pitch, start_time=note.start_time-twoSecsFromStart, end_time=note.end_time-twoSecsFromStart, velocity=80)
                elif note.start_time==twoSecsFromStart:
                    sequenceToAdd.notes.add(pitch=note.pitch, start_time=note.start_time-twoSecsFromStart, end_time=note.end_time-twoSecsFromStart, velocity=80)
                j+=1
            else:       
                if note.start_time>=twoSecsFromStart:

                    sequenceToAdd.notes.add(pitch=note.pitch, start_time=note.start_time-twoSecsFromStart, end_time=note.end_time-twoSecsFromStart, velocity=80)
    else:
        sequenceToAdd = newSequenceFull
        
    
    #adds generated sequence to larger sequence
    for note in sequenceToAdd.notes:
        
        print("notestart")
        print(note.start_time)
        sequence.notes.add(pitch=note.pitch, start_time=note.start_time+seconds_total, end_time=note.end_time+seconds_total, velocity=80)
    
    
        sequence.total_time=note.end_time+seconds_total
        
    #generates next primer dict based on last two measures of last generated sequence
        
    nextPrimer = []
    
    twoSecsFromEnd=total_seconds-(32*seconds_per_step)
    i=0
    for note in newSequenceFull.notes:
        
        if i==0:
            if note.start_time>twoSecsFromEnd:
                nextPrimer.append({'pitch':0,'length':(note.start_time-twoSecsFromEnd)*(tempo/60)})
                nextPrimer.append({'pitch':note.pitch,'length':(note.end_time-note.start_time)*(tempo/60)})
            elif note.start_time==twoSecsFromEnd:
                nextPrimer.append({'pitch':note.pitch,'length':(note.end_time-note.start_time)*(tempo/60)})       
            i+=1
        
        else:
            if note.start_time>=twoSecsFromEnd:
                
                if note.end_time<total_seconds and (i+1)==len(newSequenceFull.notes):
                    nextPrimer.append({'pitch':note.pitch,'length':(note.end_time-note.start_time)*(tempo/60)})
                    nextPrimer.append({'pitch':0,'length':(total_seconds-note.end_time)*(tempo/60)})
                else:
                    nextPrimer.append({'pitch':note.pitch,'length':(note.end_time-note.start_time)*(tempo/60)})
        
            
        
    
    
    
    counter+=1
    seconds_total=sequence.total_time
    
    print(nextPrimer)
    print("seconds total")
    print(seconds_total)
    print("measures total")
    print((seconds_total/seconds_per_step)/16)
    print("seconds per loop")
    print(total_seconds)
    return loop_generate(instrument, temperature, length, loops-1, primer_notes=nextPrimer, tempo=tempo, sequence=sequence, seconds_total=seconds_total, counter=counter)

In [16]:
generated_bass = []


#for i in range(10):
    
    #bass_seq=generate_clip("Bass", 3.5, 4, primer_notes, primer_tempo)

    

    #generated_bass.append(bass_seq)
    
bass_seq = loop_generate("Bass", 10, 4, 4, primer_notes=primer_notes, tempo=180)
print(type(bass_seq))

generated_bass.append(bass_seq)
    

i=0
for ns in generated_bass:
    note_seq.plot_sequence(ns)
    filename = "bass" + str(i)
    note_seq.note_sequence_to_midi_file(ns, filename + '.midi')
    os.rename(filename + ".midi", "./MagentaMidiSamples/" + filename + ".midi")
    loader.export_midi_file("./MagentaMidiSamples/" + filename + ".midi", name=filename + ".mp3", format = "mp3")
    os.rename(filename + ".mp3", "./MagentaSamples/" + filename + ".mp3")
    i += 1

INFO:tensorflow:Beam search yields sequence with log-likelihood: -158.996857 
notestart
0.0
notestart
0.3333333333333333
notestart
1.3333333333333333
notestart
1.4166666666666665
notestart
1.5833333333333333
notestart
1.75
notestart
1.8333333333333333
notestart
1.9166666666666665
notestart
2.0
notestart
2.083333333333333
notestart
2.1666666666666665
notestart
2.333333333333333
notestart
2.4166666666666665
notestart
2.583333333333333
notestart
2.833333333333333
notestart
3.0
notestart
3.583333333333333
notestart
3.6666666666666665
notestart
3.833333333333333
notestart
3.9166666666666665
notestart
4.0
notestart
4.083333333333333
notestart
4.25
notestart
4.333333333333333
notestart
4.416666666666666
notestart
4.5
notestart
4.583333333333333
notestart
4.666666666666666
notestart
4.75
notestart
4.833333333333333
notestart
4.916666666666666
notestart
5.0
notestart
5.083333333333333
[{'pitch': 70, 'length': 0.5000000000000009}, {'pitch': 43, 'length': 1.7499999999999991}, {'pitch': 63, 'lengt

Loading BokehJS ...

In [55]:
generated_drums = []
for i in range(10):
    

    drum_seq=generate_clip("Drums", 3, 4, primer_drums, primer_tempo)

    generated_drums.append(drum_seq)
    
j=0
for ns in generated_drums:
    note_seq.plot_sequence(ns)
    filename = "drums" + str(j)
    note_seq.note_sequence_to_midi_file(ns, filename + '.midi')
    os.rename(filename + ".midi", "./MagentaMidiSamples/" + filename + ".midi")
    loader.export_midi_file("./MagentaMidiSamples/" + filename + ".midi", name=filename + ".mp3", format = "mp3")
    os.rename(filename + ".mp3", "./MagentaSamples/" + filename + ".mp3")
    j += 1

[1, 2]
INFO:tensorflow:Beam search yields sequence with log-likelihood: -77.076698 
[soundfont loader]
loaded soundfonts: ['/home/aymliu/soundFonts/SQ_008_Rock_Bass.sf2', '/home/aymliu/soundFonts/The_Definitive_Perfect_Drums_Soundfount_V1___1-12_.sf2']
soundfonts id: [1, 2]
current channel: 0
current soundfont id: 2
current soundfont name: The_Definitive_Perfect_Drums_Soundfount_V1___1-12_.sf2
current bank number: 0
current preset number: 0
current preset name: Perfect Drums 1
[1, 2]
INFO:tensorflow:Beam search yields sequence with log-likelihood: -34.661278 
[soundfont loader]
loaded soundfonts: ['/home/aymliu/soundFonts/SQ_008_Rock_Bass.sf2', '/home/aymliu/soundFonts/The_Definitive_Perfect_Drums_Soundfount_V1___1-12_.sf2']
soundfonts id: [1, 2]
current channel: 0
current soundfont id: 2
current soundfont name: The_Definitive_Perfect_Drums_Soundfount_V1___1-12_.sf2
current bank number: 0
current preset number: 0
current preset name: Perfect Drums 1
[1, 2]
INFO:tensorflow:Beam search y

Loading BokehJS ...

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: error: There is no preset with bank number 128 and preset number 0 in SoundFont 1


Loading BokehJS ...

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: error: There is no preset with bank number 128 and preset number 0 in SoundFont 1


...

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: error: There is no preset with bank number 128 and preset number 0 in SoundFont 1


Loading BokehJS ...

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: error: There is no preset with bank number 128 and preset number 0 in SoundFont 1


Loading BokehJS ...

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: error: There is no preset with bank number 128 and preset number 0 in SoundFont 1
